# Práctica word embeddings
José María Rodríguez Cano de Santayana

In [1]:
import pandas as pd
import numpy as np
import gensim.models.word2vec as w2v
import multiprocessing
import os
import re
import pprint
import sklearn.manifold
import matplotlib.pyplot as plt
import time

unable to import 'smart_open.gcs', disabling that module


In [ ]:
#To plot in collab:

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
#call this function en cada celda que hagas un plot

### vamos a usar un dataset top_books.txt con 20 libros clásicos en texto plano.


In [2]:
with open("data/top_books.txt", "r", encoding="utf-8") as f:
    file = f.read()


file = file.replace("\n"," ")
file = file.replace(",","")
file = file.replace("…","")
file = file.replace('"','')
file = file.replace('“','')
file = file.replace('”','')
file = file.replace('?','')
file = file.replace('!','')
file = file.replace(';','')

file = file.lower()
file = file.split('.')

In [3]:
print(len(file))


text_corpus = [s.split(" ") for s in file]
#text_corpus[1:23]
print(len(text_corpus))

54443
54443


In [4]:
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7


downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [5]:
words2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

words2vec.build_vocab(text_corpus)

In [6]:
start_time = time.time()

words2vec.train(text_corpus, total_examples=words2vec.corpus_count, epochs=2)

if not os.path.exists("trained"):
    os.makedirs("trained")

words2vec.save(os.path.join("trained", "songs2vectors.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 3.0767452716827393 seconds ---


In [7]:
words2vec = w2v.Word2Vec.load(os.path.join("trained", "songs2vectors.w2v"))

#### Let's explore our model

Find similar words

In [90]:
words2vec.wv.most_similar("love")

[('mine', 0.9381674528121948),
 ('promise', 0.9375184178352356),
 ('regret', 0.935810387134552),
 ('kindness', 0.9327780604362488),
 ('care', 0.9325066208839417),
 ('trust', 0.9312163591384888),
 ('innocent', 0.930232048034668),
 ('confidence', 0.9300640225410461),
 ('to-night', 0.9277136921882629),
 ('wise', 0.9275190830230713)]

In [91]:
words2vec.wv.most_similar("cat")

[('mouse', 0.9708613753318787),
 ('smart', 0.9614558815956116),
 ('nigger', 0.9582551717758179),
 ('clever', 0.9568686485290527),
 ('cook', 0.9527952075004578),
 ('queer', 0.952789306640625),
 ('â€œitâ€™s', 0.9518734812736511),
 ('duchess', 0.9517069458961487),
 ('baby', 0.9514837265014648),
 ('thatâ€\x9d', 0.9503557682037354)]

In [92]:
words2vec.wv.most_similar("roof")

[('edge', 0.9779365658760071),
 ('team', 0.9775127172470093),
 ('trail', 0.9757276177406311),
 ('centre', 0.9755381345748901),
 ('frozen', 0.9750385880470276),
 ('by-street', 0.9737116098403931),
 ('snow', 0.9733673930168152),
 ('field', 0.9732356667518616),
 ('hearth', 0.971807062625885),
 ('angle', 0.9714497923851013)]

In [93]:
words2vec.wv.most_similar("death")

[('despair', 0.9511659145355225),
 ('happiness', 0.9442972540855408),
 ('pain', 0.9395033121109009),
 ('strength', 0.9391196966171265),
 ('existence', 0.9385036826133728),
 ('peace', 0.9364503622055054),
 ('mine', 0.9359779357910156),
 ('pleasure', 0.9349012970924377),
 ('honour', 0.9346639513969421),
 ('food', 0.9343008995056152)]

In [94]:
words2vec.wv.most_similar("hunger")

[('oppressed', 0.9830219745635986),
 ('georgiana', 0.9824586510658264),
 ('consumed', 0.980677604675293),
 ('consternation', 0.9791681170463562),
 ('shaken', 0.979145884513855),
 ('happening', 0.9787892699241638),
 ('worship', 0.9787237644195557),
 ('hardened', 0.9784822463989258),
 ('wretchedness', 0.9783862233161926),
 ('dæmon', 0.9782481789588928)]

In [95]:
 words2vec.wv.words_closer_than("man", "woman")

['gentleman', 'silver', 'fellow', 'dog']

Words out of context

In [96]:
words2vec.wv.doesnt_match("happiness love joy hate".split())

c:\users\chema\appdata\local\programs\python\python37-32\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



'hate'

In [ ]:
words2vec.wv.doesnt_match("lamp chair table plane".split())

In [ ]:
words2vec.most_similar(positive=['father', 'woman'], negative=['man'])
#queen

Semantic distance between words

In [ ]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = words2vec.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [ ]:
nearest_similarity_cosmul("fun", "game", "pain")

In [ ]:
nearest_similarity_cosmul("france", "paris", "london")

In [8]:
words2vec['right']

c:\users\chema\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-4.9188903e-01,  1.2272182e-02, -7.0339166e-02,  3.8829702e-01,
        6.7875868e-01,  8.1453890e-02,  8.5962778e-01, -8.8106953e-02,
        5.9073484e-01, -2.7534705e-01,  2.5789022e-01, -3.7279135e-01,
       -5.9430975e-01, -4.5624620e-01, -4.7404867e-01,  5.0695056e-01,
        5.3350437e-01,  1.3564733e-01,  1.3513006e-01,  1.8379822e-03,
        3.7937838e-01,  5.9776060e-02, -4.3472558e-01,  2.0514933e-02,
       -6.3318330e-01, -3.0999282e-01, -2.4280663e-01,  8.0217898e-01,
       -6.3552487e-01, -5.4308045e-01, -1.3226973e-01,  1.9002791e-01,
        2.6075539e-01, -1.0394091e-01, -1.0698275e-01, -2.9817647e-01,
        3.8800359e-01, -3.1277996e-01,  2.0564093e-01, -7.6583368e-01,
       -3.2932976e-01, -6.4192700e-01, -3.1935847e-01,  3.7386015e-02,
       -2.4061270e-02,  5.3253615e-01,  2.9653218e-04, -5.0959754e-01,
        7.9783195e-01, -7.0407861e-01], dtype=float32)

## Como no va todo lo bien que quisiera voy a unsar un datasets con word embeddings ya entrenado.
http://nlp.stanford.edu/data/glove.6B.zip <br>
https://nlp.stanford.edu/projects/glove/

In [9]:
words = []

for i,line in enumerate(open("data/glove.6B.50d.txt", "r", encoding="utf-8")):
    if i>10000:
        break
    line = line.replace("\n","")
    words.append(line.split(" "))
    

In [10]:
word_dic={}
for word in words:
    word_dic.update({
        word[0]:
        [float(e) for e in word[1:]]
    })


In [11]:
len(words)

10001

In [41]:
df_t = pd.DataFrame({"text":file})
df_t.head()

,text
0,chapter 1 it is a truth universally acknowled...
1,however little known the feelings or views o...
2,my dear mr
3,bennet said his lady to him one day have you ...
4,bennet replied that he had not


In [42]:
def get_nums(txt):
    arr = []
    for word in txt.split(" "):
        try:
            arr.append(word_dic[word])
        except KeyError:
            pass
    return arr

df_t["word_vector"] = df_t['text'].apply(get_nums)

In [44]:
def normalize(lista):
    r = np.zeros(50)
    for vec in lista:
        r += np.array(vec)
    r = sklearn.preprocessing.normalize(r.reshape(1,50))
    return r


df_t['word_vector_norm'] = df_t['word_vector'].apply(normalize)
df_t.head()

,text,word_vector,word_vector_norm
0,chapter 1 it is a truth universally acknowled...,"[[0.33036, 0.85686, -0.89838, 0.25596, 0.51197...","[[0.07227229908361216, 0.07831993910911388, -0..."
1,however little known the feelings or views o...,"[[0.43833, -0.15784, -0.12393, -0.11901, 0.146...","[[0.1035475190686613, 0.0784962769957481, -0.0..."
2,my dear mr,"[[-0.27279, 0.77515, -0.10181, -0.9166, 0.9047...","[[-0.03980275923522211, 0.21398549761345095, 0..."
3,bennet said his lady to him one day have you ...,"[[0.38973, -0.2121, 0.51837, 0.80136, 1.0336, ...","[[0.07818382793211373, 0.07892059116287511, -0..."
4,bennet replied that he had not,"[[0.064332, -0.40888, 0.038331, -0.61864, 1.14...","[[0.08379975550947732, -0.06089973236950035, -..."


In [15]:
df_t["word_vector_norm"][0].shape

(1, 50)

In [45]:
word_vectors = []
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_t, test_size = 0.9)


for word_vector in train['word_vector_norm']:
    word_vectors.append(word_vector)

train.head(4)

,text,word_vector,word_vector_norm
17057,scarce a day or night passed over but some ...,"[[0.21705, 0.46515, -0.46757, 0.10082, 1.0135,...","[[0.10093698018557816, 0.03871684121158728, -0..."
19814,) he is going,"[[-0.28314, 1.0028, 0.14746, 0.22262, 0.007098...","[[0.0082665989121334, 0.07813034833700203, -0...."
14992,poole disinterred the axe from under a stack ...,"[[0.418, 0.24968, -0.41242, 0.1217, 0.34527, -...","[[0.09254158154926734, 0.02358783118501601, -0..."
7941,his blind and aged father and his gentle sis...,"[[-0.033537, 0.47537, -0.68746, -0.72661, 0.84...","[[0.038605446697919675, 0.09889772479772638, -..."


In [46]:
len(word_vectors)

5444

In [47]:
X = np.array(word_vectors).reshape((5444, 50))

In [50]:
start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, 
                             n_iter=250, 
                             random_state=0, 
                             verbose=2)

all_word_vectors_matrix_2d = tsne.fit_transform(X)

print("--- %s seconds ---" % (time.time() - start_time))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 5444 samples in 0.050s...
[t-SNE] Computed neighbors for 5444 samples in 2.063s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5444
[t-SNE] Computed conditional probabilities for sample 2000 / 5444
[t-SNE] Computed conditional probabilities for sample 3000 / 5444
[t-SNE] Computed conditional probabilities for sample 4000 / 5444
[t-SNE] Computed conditional probabilities for sample 5000 / 5444
[t-SNE] Computed conditional probabilities for sample 5444 / 5444
[t-SNE] Mean sigma: 0.000000
[t-SNE] Computed conditional probabilities in 0.209s
[t-SNE] Iteration 50: error = 84.9144669, gradient norm = 0.0220746 (50 iterations in 0.921s)
[t-SNE] Iteration 100: error = 81.7778168, gradient norm = 0.0085871 (50 iterations in 0.847s)
[t-SNE] Iteration 150: error = 81.6593552, gradient norm = 0.0071934 (50 iterations in 0.767s)
[t-SNE] Iteration 200: error = 81.6152115, gradient norm = 0.0072066 (50 iterations in 0.720s)
[t

In [51]:
df=pd.DataFrame(all_word_vectors_matrix_2d,columns=['X','Y'])

df.head(10)

train.head()

df.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

In [52]:
two_dimensional_words = pd.concat([train, df], axis=1)

two_dimensional_words.head()

,text,word_vector,word_vector_norm,X,Y
0,scarce a day or night passed over but some ...,"[[0.21705, 0.46515, -0.46757, 0.10082, 1.0135,...","[[0.10093698018557816, 0.03871684121158728, -0...",-0.124871,-0.377172
1,) he is going,"[[-0.28314, 1.0028, 0.14746, 0.22262, 0.007098...","[[0.0082665989121334, 0.07813034833700203, -0....",0.101069,-0.012087
2,poole disinterred the axe from under a stack ...,"[[0.418, 0.24968, -0.41242, 0.1217, 0.34527, -...","[[0.09254158154926734, 0.02358783118501601, -0...",-0.182745,-0.412699
3,his blind and aged father and his gentle sis...,"[[-0.033537, 0.47537, -0.68746, -0.72661, 0.84...","[[0.038605446697919675, 0.09889772479772638, -...",-0.041991,-0.141423
4,keep it,"[[0.14817, -0.22174, 0.60221, -0.79489, 0.3345...","[[0.07997144166196589, -0.046558110628622924, ...",0.322670,0.471730


#### Es muy interesante ver como los ouliers en la siguiente gráfica son las frases sin sentido que se han colado en el dataset como "Chapter 23" u "(\_octavo\_)"

In [39]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

trace1 = go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['text'],
    mode='markers',
    marker=dict(
        size= 5,#'7',
        color = np.random.randn(5717), #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

iplot(data)

In [82]:
def dist(r):
    x1 = two_dimensional_words["X"][index]
    y1 = two_dimensional_words["Y"][index]
    return r['text'],(r["X"]-x1)**2+(r["Y"]-y1)**2

## Ahora vamos a comprobar si saber relacionar frases correctamente

In [97]:
index = 5
distances = two_dimensional_words.apply(dist, axis=1)
target = two_dimensional_words["text"][index]
closest = sorted(distances, key=lambda tup: tup[1])[1][0]

print(f"la frase más parecida a:\n'''{target}'''\n es:\n'''{closest}'''")

la frase más parecida a:
''' the fee is owed   to the owner of the project gutenberg-tm trademark but he has   agreed to donate royalties under this paragraph to the project   gutenberg literary archive foundation'''
 es:
'''  additional terms will be linked to the project gutenberg-tm license for all works posted with the permission of the copyright holder found at the beginning of this work'''


In [118]:
index = 300
distances = two_dimensional_words.apply(dist, axis=1)
target = two_dimensional_words["text"][index]
closest = sorted(distances, key=lambda tup: tup[1])[1][0]

print(f"la frase más parecida a:\n'''{target}'''\n es:\n'''{closest}'''")

la frase más parecida a:
''' lorry's spirits grew heavier and heavier as he and his two companions ascended higher and higher'''
 es:
''' his were the shinbones of the saline beef his would have been the drumsticks'''


mas o menos...